<a href="https://colab.research.google.com/github/markiroberts/gogglecolab01/blob/main/03_pt02_Use_matches_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import data_table

data_table.enable_dataframe_formatter()

directory_path = "/content/drive/MyDrive/ColabData/"
try:
  os.stat(directory_path)
except:
  print(f"Cant access {directory_path}")
  drive.mount('/content/drive')

match_df = pd.read_csv("/content/drive/MyDrive/ColabData/matches.csv")

match_df.head()



,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,formation,referee,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Away,L,1.0,2.0,...,3-4-3,Michael Oliver,6.0,2.0,17.3,0.0,0,0,2023,Nottingham Forest
1,1,2023-08-18,19:45,Premier League,Matchweek 2,Fri,Home,W,2.0,1.0,...,3-4-3,Peter Bankes,16.0,4.0,15.6,0.0,0,0,2023,Nottingham Forest
2,2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Away,L,2.0,3.0,...,3-4-3,Stuart Attwell,9.0,4.0,14.7,0.0,0,0,2023,Nottingham Forest
3,3,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,3-4-3,Tim Robinson,7.0,3.0,19.0,0.0,0,0,2023,Nottingham Forest
4,4,2023-09-18,19:45,Premier League,Matchweek 5,Mon,Home,D,1.0,1.0,...,4-2-3-1,Robert Jones,14.0,4.0,18.8,0.0,0,0,2023,Nottingham Forest


In [125]:
#df = pd.DataFrame({'B': [0, 1, 2, 3, 4]})
#df['Bavg2'] = df.B.shift(1).rolling(window=2).mean()
#df = df.dropna(subset='Bavg2')
#https://fbref.com/en/comps/9/Premier-League-Stats

teamdict = {
"Wolverhampton Wanderers": "Wolves",
"Brighton and Hove Albion": "Brighton",
"West Ham United": "West Ham",
"Tottenham Hotspur": "Tottenham",
"Manchester United": "Manchester Utd",
"Newcastle United": "Newcastle Utd",
"Sheffield United": "Sheffield Utd",
"Nottingham Forest": "Nott'ham Forest"
}

def getTeam(team):
  opponent = teamdict.get(team,None)
  if opponent:
    return opponent
  else:
    return team


match_df = pd.read_csv("/content/drive/MyDrive/ColabData/matches.csv")
match_df.head()

# the team name an opponent list are initially not the same !?
# translate team to the equivalent from oponent list 'Sheffield United -> Sheffield Utd.' etc.
match_df['team'] = match_df.apply(lambda row: getTeam(row.team), axis=1)
teamset = sorted(set(match_df['team'].tolist()))
opponentset = sorted(set(match_df['opponent'].tolist()))
allmatch_team = pd.DataFrame()

for team in teamset:
  options = []
  options.append(team)

# selecting rows based on condition

  match_team = match_df[match_df['team'].isin(options)]
  selection = ['result','venue']
  match_team = pd.get_dummies(match_team, columns=selection)
  match_team['dateandtime'] = pd.to_datetime( match_team['date'] + ' ' + match_team['time'])
  match_team['hour'] = match_team['dateandtime'].dt.hour
  match_team['result'] = 1.0 * match_team['result_W'] - 1.0 * match_team['result_L']
  match_team.sort_values("dateandtime")


  match_team['result_1']   = match_team.result.shift(1).rolling(window=1).mean()
  match_team['result_L_1'] = match_team.result_L.shift(1).rolling(window=1).mean()
  match_team['result_D_1'] = match_team.result_D.shift(1).rolling(window=1).mean()
  match_team['result_W_1'] = match_team.result_W.shift(1).rolling(window=1).mean()
  match_team['gf_1'] = match_team.gf.shift(1).rolling(window=1).mean()
  match_team['ga_1'] = match_team.ga.shift(1).rolling(window=1).mean()

  match_team['result_L_3'] = match_team.result_L.shift(1).rolling(window=3).mean()
  match_team['result_D_3'] = match_team.result_D.shift(1).rolling(window=3).mean()
  match_team['result_W_3'] = match_team.result_W.shift(1).rolling(window=3).mean()
  match_team['gf_3'] = match_team.gf.shift(1).rolling(window=3).mean()
  match_team['ga_3'] = match_team.ga.shift(1).rolling(window=3).mean()

  match_team['result_3']   = match_team.result.shift(1).rolling(window=3).mean()
  match_team['result_L_5'] = match_team.result_L.shift(1).rolling(window=5).mean()
  match_team['result_D_5'] = match_team.result_D.shift(1).rolling(window=5).mean()
  match_team['result_W_5'] = match_team.result_W.shift(1).rolling(window=5).mean()
  match_team['gf_5'] = match_team.gf.shift(1).rolling(window=5).mean()
  match_team['ga_5'] = match_team.ga.shift(1).rolling(window=5).mean()

  match_team['result_5']   = match_team.result.shift(1).rolling(window=5).mean()
  match_team = match_team.dropna()

  matchedcols = []
  columnlist = ['dateandtime','hour','result','result', 'gf', 'ga', 'day', 'xg', 'hour', 'team', 'opponent','season']
  for findcol in match_team.columns:
    for matchcol in columnlist:
      if findcol.startswith(matchcol):
        if findcol not in matchedcols:
          matchedcols.append(findcol)
  newmatch_team = match_team[matchedcols]
  allmatch_team = pd.concat([allmatch_team,newmatch_team], ignore_index=True)


allmatch_team = allmatch_team.sort_values(by=['team','dateandtime'], ignore_index=True)

In [126]:
allmatch_team[['team','season','gf','gf_1','gf_3','result']]

,team,season,gf,gf_1,gf_3,result
0,Arsenal,2021,0.0,5.0,1.666667,-1.0
1,Arsenal,2021,0.0,0.0,1.666667,-1.0
2,Arsenal,2021,0.0,0.0,1.666667,-1.0
3,Arsenal,2021,1.0,0.0,0.000000,1.0
4,Arsenal,2021,1.0,1.0,0.333333,1.0
...,...,...,...,...,...,...
1750,Wolves,2023,1.0,2.0,1.666667,-1.0
1751,Wolves,2023,1.0,1.0,1.666667,1.0
1752,Wolves,2023,1.0,1.0,1.333333,0.0
1753,Wolves,2023,0.0,1.0,1.000000,-1.0
